In [13]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [14]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"

    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "round summary" in dict.keys():
        # add round summary as a memory to the metadata
        # it will be used in the next round.
        config.metadata["memories"].append(f"ROUND {current_round}: {dict["round summary"]}")
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    
    
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_36.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


You arrived at an old self-storage facility by invitation from the manager, who has an storage room up for auction due to the death of its owner. It is not your first rodeo with these forms of auctions, like a box of chocolate, no one knows of the potential treasure that hides behind the cold roller shutter. As you and other interested buyers gathered around the auctioneer, a worker carrying a bolt cutter snapped off the rusty lock, unveiling a storage room cluttered with various old furniture and antiquities. Without any opportunity for the bidders to take a clear look, the auctioneer began the bid at $50. 

What is your bid?

possible actions:
1. $100.
2. $150.
3. $200.


I select option 2. TURN: 1 


You raise your hand confidently, bidding $150. The auctioneer nods in your direction, acknowledging your bid. The crowd murmurs among themselves, sizing up the storage unit and its visible contents from their vantage points. Another bidder, a woman in her mid-50s with a keen eye for antiques, quickly counters with a $200 bid. The auctioneer's voice rises over the crowd, announcing the new bid.

possible actions:
1. Increase your bid to $250.
2. Increase your bid to $300.
3. Withdraw from the bidding and observe.

I select option 1. TURN: 2 


You decisively increase your bid to $250, catching the attention of the other bidders. The auctioneer points towards you, "We have $250 from the gentleman over here! Do I hear $300?" The atmosphere grows tense as bidders whisper to each other, calculating their next moves. The woman who previously outbid you hesitates, then folds her arms, stepping back from the competition. No one else seems eager to challenge your bid, and the auctioneer's voice fills the silence, "Going once... Going twice..."

possible actions:
1. Maintain your bid at $250 and wait for the auctioneer's final call.
2. Proactively increase your bid to discourage other potential bidders.
3. Withdraw your bid and observe, taking a chance on the next storage unit.

I select option 1. TURN: 3 


Maintaining your bid at $250, you watch as the auctioneer scans the crowd for any last-minute bids. The tension is palpable, with every bidder holding their breath, waiting for the final call. The woman who had previously challenged you remains silent, her interest apparently waned. No new hands rise, and the auctioneer's voice booms, "Sold to the gentleman for $250!"

Congratulations, you've won the auction for the storage unit.

round summary: Kyle Jackson won the auction for the storage unit with a bid of $250, successfully acquiring potential treasures hidden within without exhausting his budget.


Memories:
"""
- ROUND 0: Kyle Jackson won the auction for the storage unit with a bid of $250, successfully acquiring potential treasures hidden within without exhausting his budget.
"""


You follow the manager through the myriad hallways to his dimly lit office, where the furnishings and appliances remain unchanged since the 90’s. He often reminds you that the facility is in fact even older, and has been here since the 60’s. You made a quick phone call to Alex, a man you owe money to, and explained to him that you will pay him back in a few days once you have it, to which he responded with distasteful threats. The manager took out an old tin box and placed it on his desk and reached towards you with an open palm, gesturing for you to pay up, which you obliged. Upon handing you a key for your newly owned storage room, he urged you to watch some old security footage.

Through his CRT screen, he explains to you that the previous owner visited the storage room every day for as long as h

...

I select option 1. TURN: 2 


skill: library_use
difficulty: easy

I am making a skill check using LIBRARY_USE against a difficulty of EASY.
And I rolled a 83 for a result of FAILURE. TURN: 3 


As you approach the collection of old, leather-bound books, you're immediately taken by their ancient appearance. The titles are in languages you can't decipher, some etched in what appears to be gold leaf, while others are too faded to read. Despite your efforts to make sense of them, your lack of familiarity with these languages and the obscure subject matter leaves you puzzled. You carefully pull one from the shelf, but the text inside is just as enigmatic, filled with strange symbols and illustrations that seem to dance before your eyes in the dim light.

Frustrated by your inability to glean any useful information, you place the book back on the shelf, feeling a bit disheartened. The failure to understand these potentially valuable texts weighs on you, but the storage unit holds more mysteries to uncover.

What would you like to do next?

possible actions

I select option 2. TURN: 4 


skill: spot_hidden
difficulty: medium

I am making a skill check using SPOT_HIDDEN against a difficulty of MEDIUM.
And I rolled a 92 for a result of FAILURE. TURN: 5 


You turn your attention to the old-fashioned radio, its wooden casing polished yet dulled by time, with knobs and switches that look alien in their antiquity. You reach out, feeling the cold, smooth surface under your fingertips, and with a mix of curiosity and reverence, you attempt to examine it closely, hoping to uncover something hidden or perhaps a clue as to why it was kept here.

However, your efforts to find anything out of the ordinary about the radio are in vain. Despite your careful examination, it seems to be just an old, albeit well-crafted, radio. The dials turn with a satisfying click, but without electricity in the storage unit, you can't test its functionality. The sense of mystery that initially enveloped the radio dissipates, leaving you feeling as though you've missed something important, something just beyond your grasp.

As you stand th

I select option 1. TURN: 6 


The round has ended, and no further actions can be taken at this moment. Please await the beginning of the next round for further exploration and decisions.